In [1]:
from pycocotools.coco import COCO
import json

In [2]:
object_annotation_path = "dataset/annotations/instances_train2017.json"

In [ ]:
object_annotations = COCO(annotation_file=object_annotation_path)

loading annotations into memory...


In [ ]:
imageIds = object_annotations.getImgIds()

In [ ]:
with open("IdLookup.txt", "r") as f:
    lookup = json.loads(f.read())

In [ ]:
imagesAndObjects = {}
objectsInDataset ={}
backgroundImages = []

for image in imageIds:
    imageAnnotations = object_annotations.loadAnns(object_annotations.getAnnIds(image, iscrowd=None))
    objects = []
    for label in imageAnnotations:
        objects.append(lookup[str(label['category_id'])])
        
    if(objects == []):
        backgroundImages.append(image)
    else:
        imagesAndObjects[str(image)] = objects

# Images per class. ≥ 1500 images per class recommended
# Instances per class. ≥ 10000 instances (labeled objects) per class recommended
# . Background images are images with no objects that are added to a dataset to reduce False Positives (FP). We recommend about 0-10% background images to help reduce FPs

In [ ]:
imagesAndObjects

In [ ]:
backgroundImages

In [ ]:
#We add all the background images and check if they are not larger than 10% of the dataset.
imagesForDataset = {}
for i in backgroundImages:
    imagesForDataset[str(i)] = ['background']


In [ ]:
imagesForDataset

In [ ]:
len(imagesForDataset)

In [ ]:
cat_ids = object_annotations.getCatIds()
cats = object_annotations.loadCats(cat_ids)

In [ ]:
cats

In [ ]:
#Create a list of every object where we count how many instances are in the dataset.
instancesInDataset = {}
for i in cats:
    instancesInDataset[i['name']] = 0
    
#Get all images containing the object for each object.
for cat in cats:
    imageId = object_annotations.getImgIds(catIds=[cat['id']])
    for image in imageId:
        #We stop at 15000 instances.
        if instancesInDataset[cat['name']] > 15000:
            print(f"Done working on {[cat['name']]}")
            break
        if str(image) in imagesForDataset:
            continue
        #If the image passes all the checks we add the image to the dataset.
        #We increment all the object intances found in the image.
        imageAnnotations = object_annotations.loadAnns(object_annotations.getAnnIds(image, iscrowd=None))
        imagesForDataset[str(image)] = imagesAndObjects[str(image)]
        for label in imageAnnotations:
            instancesInDataset[lookup[str(label['category_id'])]] = instancesInDataset[lookup[str(label['category_id'])]] + 1
        

In [ ]:
imagesForDataset

In [ ]:
len(imagesForDataset)

In [ ]:
instancesInDataset

In [ ]:
#We need to rearrange the order of these so that the once with the smallest numbers are done first.
sortedDict = dict(sorted(instancesInDataset.items(), key=lambda item: item[1]))

In [ ]:
#Now we redo the loop except will will add the items to the list of images to see if there is a decrease.
imagesForDataset = {}
for i in backgroundImages:
    imagesForDataset[str(i)] = ['background']
    
#instead of using cats as our main loop we will use instancesInDataset which is a sorted dictionary(We will first dump the contents into a list)
newInstancesInDataset = {}
for key in sortedDict:
    newInstancesInDataset[key] = 0
    
print(newInstancesInDataset)
for cat in newInstancesInDataset:    
    imageId = object_annotations.getImgIds(catIds=[object_annotations.getCatIds(catNms=[cat])[0]])
    for image in imageId:
        #We stop at 15000 instances.
        if newInstancesInDataset[cat] > 15000:
            print(f"Done working on {cat}")
            break
        if str(image) in imagesForDataset:
            continue
        #If the image passes all the checks we add the image to the dataset.
        #We increment all the object intances found in the image.
        imageAnnotations = object_annotations.loadAnns(object_annotations.getAnnIds(image, iscrowd=None))
        imagesForDataset[str(image)] = imagesAndObjects[str(image)]
        for label in imageAnnotations:
            newInstancesInDataset[lookup[str(label['category_id'])]] = newInstancesInDataset[lookup[str(label['category_id'])]] + 1

In [ ]:
print(len(imagesForDataset))

In [ ]:
#This sucks